<a href="https://colab.research.google.com/github/pawareliza14/TicTacToe-Neural-Network/blob/main/tictactoe_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Generate more realistic synthetic data
def generate_realistic_data(n_samples=20000):
    # Create empty boards
    X = np.zeros((n_samples, 9))
    y = np.zeros(n_samples)

    for i in range(n_samples):
        # Random number of moves already on board (0-8)
        moves = np.random.randint(0, 5)  # Limited to avoid full boards
        board = np.zeros(9)

        # Place random X and O moves
        positions = np.random.choice(9, moves, replace=False)
        for j, pos in enumerate(positions):
            board[pos] = 1 if j % 2 == 0 else -1

        # For empty positions, determine best move using a heuristic
        empty_positions = np.where(board == 0)[0]
        if len(empty_positions) > 0:
            # Simple heuristic: prefer center, then corners, then sides
            priority_positions = [4, 0, 2, 6, 8, 1, 3, 5, 7]
            for pos in priority_positions:
                if pos in empty_positions:
                    best_move = pos
                    break
            y[i] = best_move
            X[i] = board

    return X, y


In [7]:

# Generate and prepare data
X, y = generate_realistic_data()
y_encoded = keras.utils.to_categorical(y, num_classes=9)

# Split data with a validation set
X_train, X_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Data normalization
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0) + 1e-8  # Add small epsilon to avoid division by zero
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std
X_test = (X_test - X_mean) / X_std


In [8]:

# Build an improved model
model = Sequential([
    # Input layer
    Dense(128, input_shape=(9,), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.1),
    Dropout(0.3),

    # Hidden layer 1
    Dense(128, kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.1),
    Dropout(0.3),

    # Hidden layer 2
    Dense(64, kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.1),
    Dropout(0.2),

    # Output layer
    Dense(9, activation='softmax')
])

# Use a fixed learning rate instead of a schedule
# This resolves the conflict with ReduceLROnPlateau
optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add callbacks for training optimization
# Removed ReduceLROnPlateau since it conflicts with learning rate schedules
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

# Train with validation data
history = model.fit(
    X_train, y_train,
    epochs=100,  # We'll stop early with callbacks
    batch_size=64,  # Larger batch size
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# Save the model
model.save("tictactoe_best_move_optimized.h5")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7249 - loss: 1.2305 - val_accuracy: 0.9827 - val_loss: 0.3321
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9658 - loss: 0.3313 - val_accuracy: 0.9913 - val_loss: 0.2050
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9823 - loss: 0.2402 - val_accuracy: 0.9910 - val_loss: 0.1625
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9845 - loss: 0.1881 - val_accuracy: 0.9980 - val_loss: 0.1261
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9897 - loss: 0.1445 - val_accuracy: 0.9983 - val_loss: 0.0948
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9938 - loss: 0.1116 - val_accuracy: 0.9990 - val_loss: 0.0758
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9942 - loss: 0.0899 - val_accuracy: 0.9990 - val_loss: 0.0584
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9951 - loss: 0.0701 - val_accuracy: 0.9990

Test accuracy: 0.9983


In [9]:

# Improved prediction function
def predict_move(board_state, model, valid_moves_only=True):
    """
    Predict the best move given a board state

    Args:
        board_state: List or array of 9 elements (1=X, -1=O, 0=empty)
        model: Trained TicTacToe model
        valid_moves_only: If True, only considers empty spaces as valid moves

    Returns:
        The index of the best move (0-8)
    """
    # Ensure board_state is numpy array
    board_state_np = np.array(board_state).reshape(1, -1)

    # Store original board for move validation
    original_board = np.array(board_state).reshape(-1)

    # Normalize the input
    board_state_normalized = (board_state_np - X_mean) / X_std

    # Get move probabilities
    move_probs = model.predict(board_state_normalized, verbose=0)[0]

    # Only consider empty positions if requested
    if valid_moves_only:
        for i in range(9):
            if original_board[i] != 0:  # Position already taken
                move_probs[i] = -np.inf

    return np.argmax(move_probs)

# Example usage
board_state = [0, 0, 0, 0, 0, 0, 0, 0, 0]  # Empty board
best_move = predict_move(board_state, model)
print(f"Best Move: {best_move}")

# Visualize a board state and prediction
def print_board(board_state):
    symbols = {0: ' ', 1: 'X', -1: 'O'}
    print('-' * 13)
    for i in range(0, 9, 3):
        print(f"| {symbols[board_state[i]]} | {symbols[board_state[i+1]]} | {symbols[board_state[i+2]]} |")
        print('-' * 13)

# Example with a more realistic board
example_board = [1, 0, -1, 0, 1, 0, 0, -1, 0]
print("Current board:")
print_board(example_board)
best_move = predict_move(example_board, model)
print(f"Recommended move: {best_move}")

Best Move: 4
Current board:
-------------
| X |   | O |
-------------
|   | X |   |
-------------
|   | O |   |
-------------
Recommended move: 6


In [11]:
def human_vs_ai(model):
    board = [0] * 9  # Empty board
    symbols = {0: ' ', 1: 'X', -1: 'O'}

    def print_board(board):
        print('-' * 13)
        for i in range(0, 9, 3):
            print(f"| {symbols[board[i]]} | {symbols[board[i+1]]} | {symbols[board[i+2]]} |")
            print('-' * 13)

    def check_winner(b):
        win_states = [
            [0,1,2], [3,4,5], [6,7,8],  # rows
            [0,3,6], [1,4,7], [2,5,8],  # cols
            [0,4,8], [2,4,6]            # diagonals
        ]
        for state in win_states:
            line = [b[i] for i in state]
            if sum(line) == 3:
                return 1  # X wins
            elif sum(line) == -3:
                return -1  # O wins
        if 0 not in b:
            return 0  # Draw
        return None  # Game ongoing

    # Choose side
    player_symbol = input("Do you want to be X or O? (X goes first): ").upper()
    while player_symbol not in ['X', 'O']:
        player_symbol = input("Please enter 'X' or 'O': ").upper()
    player = 1 if player_symbol == 'X' else -1
    ai = -player

    turn = 1  # 1 = X's turn, -1 = O's turn
    print("\nStarting game!")

    while True:
        print_board(board)
        if turn == player:
            # Human move
            try:
                move = int(input("Enter your move (0-8): "))
                if move < 0 or move > 8 or board[move] != 0:
                    print("Invalid move. Try again.")
                    continue
            except ValueError:
                print("Invalid input. Enter a number from 0 to 8.")
                continue
        else:
            # AI move
            print("AI is thinking...")
            move = predict_move(board, model)

        board[move] = turn
        winner = check_winner(board)
        if winner is not None:
            print_board(board)
            if winner == 0:
                print("It's a draw!")
            elif winner == player:
                print("You win! 🎉")
            else:
                print("AI wins! 🤖")
            break

        turn *= -1  # Switch turns


In [12]:
human_vs_ai(model)


Do you want to be X or O? (X goes first): O

Starting game!
-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
AI is thinking...
-------------
|   |   |   |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
Enter your move (0-8): 0
-------------
| O |   |   |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
AI is thinking...
-------------
| O |   | X |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
Enter your move (0-8): 3
-------------
| O |   | X |
-------------
| O | X |   |
-------------
|   |   |   |
-------------
AI is thinking...
-------------
| O |   | X |
-------------
| O | X |   |
-------------
| X |   |   |
-------------
AI wins! 🤖
